In [9]:
from ngraph.scenario import Scenario
from ngraph.traffic_demand import TrafficDemand
from ngraph.traffic_manager import TrafficManager
from ngraph.lib.flow_policy import FlowPolicyConfig, FlowPolicy, FlowPlacement
from ngraph.lib.algorithms.base import PathAlg, EdgeSelect
from ngraph.failure_manager import FailureManager
from ngraph.failure_policy import FailurePolicy, FailureRule, FailureCondition
from ngraph.explorer import NetworkExplorer

In [10]:
scenario_yaml = """
blueprints:
  server_pod:
    rsw:
        node_count: 48
        attrs:
          hw_component: Minipack2_128x200GE
  
  f16_2tier:
    groups:
      ssw:
        node_count: 36
        attrs:
          hw_component: Minipack2_128x200GE
      fsw:
        node_count: 96
        attrs:
          hw_component: Minipack2_128x200GE

    adjacency:
      - source: /ssw
        target: /fsw
        pattern: mesh
        link_params:
          capacity: 200
          cost: 1
          
  hgrid_2tier:
    groups:
      fauu:
        node_count: 8
        attrs:
          hw_component: Minipack2_128x200GE
      fadu:
        node_count: 36
        attrs:
          hw_component: Minipack2_128x200GE

    adjacency:
      - source: /fauu
        target: /fadu
        pattern: mesh
        link_params:
          capacity: 400
          cost: 1

  fa:
    groups:
      fa[1-16]:
        use_blueprint: hgrid_2tier
                  
  dc_fabric:
    groups:
      plane[1-8]:
        use_blueprint: f16_2tier

      pod1:
        use_blueprint: server_pod
      pod36:
        use_blueprint: server_pod
    
    adjacency:
        - source: /pod1/rsw
          target: /plane[0-9]*/fsw/fsw-1
          pattern: mesh
          link_params:
            capacity: 200
            cost: 1
        - source: /pod36/rsw
          target: /plane[0-9]*/fsw/fsw-36
          pattern: mesh
          link_params:
            capacity: 200
            cost: 1

  ebb:
    groups:
      eb0[1-8]:
        node_count: 4 

    adjacency:
      - source: "eb0{idx}"
        target: "eb0{idx}"
        expand_vars:
          idx: [1, 2, 3, 4, 5, 6, 7, 8]
        expansion_mode: "zip"
        pattern: "mesh"
        link_params: 
          capacity: 3200
          cost: 10
             
network:
  name: "fb_region"
  version: 1.0

  groups:
    dc[1-3, 5-6]:
      use_blueprint: dc_fabric

    fa:
        use_blueprint: fa

    ebb:
        use_blueprint: ebb

  adjacency:
        - source: ".*/ssw/"
          target: ".*/fa{fa_id}/fadu"
          expand_vars:
            fa_id: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
          pattern: one_to_one
          link_params:
            capacity: 200
            cost: 1
        - source: .*/fauu-[15]
          target: .*/eb0[1-8]-1
          pattern: mesh
          link_count: 2
          link_params:
            capacity: 400
            cost: 1
        - source: .*/fauu-[26]
          target: .*/eb0[1-8]-2
          pattern: mesh
          link_count: 2
          link_params:
            capacity: 400
            cost: 1   
        - source: .*/fauu-[37]
          target: .*/eb0[1-8]-3
          pattern: mesh
          link_count: 2
          link_params:
            capacity: 400
            cost: 1 
        - source: .*/fauu-[48]
          target: .*/eb0[1-8]-4
          pattern: mesh
          link_count: 2
          link_params:
            capacity: 400
            cost: 1  
components:
  Minipack2_128x200GE:
    component_type: router
    power_watts: 1750  # typical power consumption with 128x200GE QSFP56 200G-FR4 at 30C
  QSFP56_200G-FR4_2km:
    component_type: pluggable_optics
    power_watts: 6.5  
"""
scenario = Scenario.from_yaml(scenario_yaml)
network = scenario.network

In [11]:
network.nodes["dc1/plane1/ssw/ssw-1"]

Node(name='dc1/plane1/ssw/ssw-1', attrs={'hw_component': 'Minipack2_128x200GE', 'type': 'node', 'disabled': False})

In [12]:
comp_lib = scenario.components_library
comp_lib.get("Minipack2_128x200GE")

Component(name='Minipack2_128x200GE', component_type='router', description='', cost=0.0, power_watts=1750.0, power_watts_max=0.0, capacity=0.0, ports=0, count=1, attrs={}, children={})

In [13]:
network.max_flow(
    source_path=".*/fsw.*",
    sink_path=".*/eb.*",
    mode="combine",
    shortest_path=True,
)

{('.*/fsw.*', '.*/eb.*'): 819200.0}

In [14]:
# Profiling
import cProfile
import pstats

profiler = cProfile.Profile()
profiler.enable()

network.max_flow(
    source_path=".*/fsw.*",
    sink_path=".*/eb.*",
    mode="combine",
    shortest_path=True,
)

profiler.disable()

stats = pstats.Stats(profiler)
stats.sort_stats(pstats.SortKey.TIME).print_stats()

         6762378 function calls (6746975 primitive calls) in 2.177 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   339840    0.399    0.000    1.184    0.000 /Users/networmix/ws/NetGraph/ngraph/lib/graph.py:143(add_edge)
   339840    0.253    0.000    0.269    0.000 /Users/networmix/ws/NetGraph/ngraph-venv/lib/python3.13/site-packages/networkx/classes/coreviews.py:81(__getitem__)
        1    0.251    0.251    0.274    0.274 /Users/networmix/ws/NetGraph/ngraph/lib/algorithms/spf.py:94(_spf_fast_all_min_cost_with_cap_remaining_dijkstra)
   339840    0.237    0.000    0.327    0.000 /Users/networmix/ws/NetGraph/ngraph-venv/lib/python3.13/site-packages/networkx/classes/multidigraph.py:417(add_edge)
        1    0.165    0.165    0.189    0.189 /Users/networmix/ws/NetGraph/ngraph/lib/algorithms/calc_capacity.py:10(_init_graph_data)
        1    0.155    0.155    1.281    1.281 /Users/networmix/ws/NetGraph/ngraph/network.py:1

In [15]:
explorer = NetworkExplorer.explore_network(network, scenario.components_library)

In [16]:
explorer.print_tree(skip_leaves=True, detailed=True)

- root | Nodes=6016, Links=167984, Cost=0.0, Power=10472000.0 | IntLinkCap=35072000.0, ExtLinkCap=0.0
  - dc1 | Nodes=1056, Links=32256, Cost=0.0, Power=1848000.0 | IntLinkCap=5529600.0, ExtLinkCap=921600.0
     -> External to [fa/fa1/fadu]: 288 links, cap=57600.0
     -> External to [fa/fa10/fadu]: 288 links, cap=57600.0
     -> External to [fa/fa11/fadu]: 288 links, cap=57600.0
     -> External to [fa/fa12/fadu]: 288 links, cap=57600.0
     -> External to [fa/fa13/fadu]: 288 links, cap=57600.0
     -> External to [fa/fa14/fadu]: 288 links, cap=57600.0
     -> External to [fa/fa15/fadu]: 288 links, cap=57600.0
     -> External to [fa/fa16/fadu]: 288 links, cap=57600.0
     -> External to [fa/fa2/fadu]: 288 links, cap=57600.0
     -> External to [fa/fa3/fadu]: 288 links, cap=57600.0
     -> External to [fa/fa4/fadu]: 288 links, cap=57600.0
     -> External to [fa/fa5/fadu]: 288 links, cap=57600.0
     -> External to [fa/fa6/fadu]: 288 links, cap=57600.0
     -> External to [fa/fa7/fadu